In [93]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [94]:
dados.shape

(10000, 4)

In [95]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score


In [96]:
x = dados[['preco','idade_do_modelo','km_por_ano']]
y = dados['vendido']

.
SEED = 158020
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [97]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier()
dummy.fit(treino_x, treino_y)
acuracia = (dummy.score(teste_x,teste_y)*100).round(2)
print(f'Acurácia do Dummy: {acuracia}% ')

Acurácia do Dummy: 58.0% 


In [98]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


Alterando o SEED altera-se a acurácia

## Validação Cruzada

In [99]:
from sklearn.model_selection import cross_validate
SEED = 158020
modelo = DecisionTreeClassifier(max_depth=2)
resultados = cross_validate(modelo,x,y,cv = 5)
media = resultados['test_score'].mean()
desv_pad = resultados['test_score'].std()
print(f'Intervalo de acurácia: {((media - 2*desv_pad)*100).round(2), ((media + 2*desv_pad)*100).round(2)}')

Intervalo de acurácia: (75.21, 76.35)


### Aleatoriedade no cross validate

In [100]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desv_pad = results['test_score'].std()
    print(f'Média da acurácia: {(media*100).round(2)} %')
    print(f'Intervalo da acurácia: {((media - 2*desv_pad)*100).round(2), ((media + 2*desv_pad)*100).round(2)}')

In [101]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv = cv)
media = resultados['test_score'].mean()
desv_pad = resultados['test_score'].std()
imprime_resultados(results)

Média da acurácia: 75.78 %
Intervalo da acurácia: (74.37, 77.19)


In [102]:
SEED = 301
np.random.seed(SEED)
cv = KFold(n_splits=5, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv = cv)
media = resultados['test_score'].mean()
desv_pad = resultados['test_score'].std()
imprime_resultados(results)

Média da acurácia: 75.76 %
Intervalo da acurácia: (73.46, 78.06)


## Simulando situação de azar / proporção desbalanceada

In [103]:
dados_azar = dados.sort_values('vendido',ascending=True)
x_azar = dados_azar[['preco','idade_do_modelo','km_por_ano']]
y_azar = dados_azar['vendido']


In [104]:
# sem shuffle
cv = KFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x_azar,y_azar,cv = cv)
media = resultados['test_score'].mean()
desv_pad = resultados['test_score'].std()
imprime_resultados(results)

Média da acurácia: 57.84 %
Intervalo da acurácia: (34.29, 81.39)


In [105]:
# com shuffle
SEED = 301
np.random.seed(SEED)
cv = KFold(n_splits=5, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x_azar,y_azar,cv = cv)
media = resultados['test_score'].mean()
desv_pad = resultados['test_score'].std()
imprime_resultados(results)

Média da acurácia: 75.78 %
Intervalo da acurácia: (73.35, 78.21)


In [106]:
# usando o stratified
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=10, shuffle=True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x_azar,y_azar,cv = cv)
media = resultados['test_score'].mean()
desv_pad = resultados['test_score'].std()
imprime_resultados(results)

Média da acurácia: 75.78 %
Intervalo da acurácia: (72.93, 78.63)


## Gerando dados aleatórios

In [107]:
np.random.seed(SEED)
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2,3,size=10000) 
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1


In [108]:
dados.head(10)

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5
5,55405.26,1,11,23594.53374,13
6,93415.61,1,16,16077.30660,20
7,65265.09,0,10,18666.73466,14
8,43917.53,1,19,12607.56956,19
9,107860.41,1,15,24000.08742,15


## Validação cruzada usando grupos

In [111]:
from sklearn.model_selection import GroupKFold
SEED = 301
np.random.seed(SEED)
cv = GroupKFold(n_splits=10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x_azar,y_azar,cv = cv, groups=dados.modelo)
media = resultados['test_score'].mean()
desv_pad = resultados['test_score'].std()
imprime_resultados(results)

Média da acurácia: 75.78 %
Intervalo da acurácia: (73.67, 77.9)


## Validação cruzada com escaldor

In [114]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

scaler = StandardScaler()
scaler.fit(x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)


In [118]:
SEED = 301
np.random.seed(SEED)
modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)
acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)


A acurácia foi 74.36%


In [120]:

SEED = 301
np.random.seed(SEED)
cv = GroupKFold(n_splits=10)

modelo = SVC()
results = cross_validate(modelo,x_azar,y_azar,cv = cv, groups=dados.modelo)

imprime_resultados(results)

Média da acurácia: 77.27 %
Intervalo da acurácia: (74.35, 80.2)


In [122]:
from sklearn.pipeline import Pipeline


SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler),('estimador', modelo)])

cv = GroupKFold(n_splits=10)
modelo = SVC()
results = cross_validate(pipeline,x_azar,y_azar,cv = cv, groups=dados.modelo)

imprime_resultados(results)
pipeline

Média da acurácia: 76.68 %
Intervalo da acurácia: (74.28, 79.08)


Pipeline(steps=[('transformacao', StandardScaler()), ('estimador', SVC())])